In [1]:
import pandas as pd
import subprocess
def process_file(input_file):
    infile = open(input_file,'rt')
    columns = []
    rows = []
    lines = []
    for line in infile.readlines():#uses all variants in the dataset. This includes "RM fail" 
        #print(line)
        if line.startswith("#"):
            #print(line[1::])
            for element in enumerate(line[1::].split()):
                rows.append(element[0])
                columns.append(element[1])
        else:
            lines.append(line.split())
        df = pd.DataFrame(lines,columns=list(columns))
        pandas_columns = len(df.columns.tolist())
    print(df.shape)
    infile.close()
    return df
    

In [2]:
i = 0

def samtools_extract(path,chrom, left_bound, right_bound,mod):
    cmd = f"samtools faidx {path} {mod} {chrom}:{left_bound}-{right_bound}"
    #print(cmd)
    #subprocess.run(cmd,shell=True)
    proc = subprocess.run(cmd,shell=True, encoding='utf-8', stdout=subprocess.PIPE)
    seq = ""
    for line in proc.stdout.split('\n'):
        if line.startswith(">"):
            header = line
        else:
            seq = seq + line
    seq =  seq.lower()
    return header, seq

def scanning(sequence):
    scan_length=15
    
    max_length = len(sequence)+1
    max_a = 0
    for i in range(0,max_length-scan_length):
        #print(sequence[i:i+scan_length])
        current_a_percent = sequence[i:i+scan_length].count("a")
        #print(current_a_percent)
        if max_a < current_a_percent:
            max_a = current_a_percent
    return max_a
def process_sequence(sequence):
    max_a = scanning(sequence)
    #print(max_a)
    
    
    return max_a
def process_dict(dict_of_loci):
    total = 0
    long_poly = 0
    long_poly_and_TSD = 0
    for key in dict_of_loci.keys():
        process_dict = dict_of_loci[key]
        total+=1
        if process_dict['max'] >= 12 or process_dict['Adjacent_Poly_length'] >= 10:
            long_poly +=1
            if process_dict['TSD_len'] >= 10:
                long_poly_and_TSD +=1
    print(total)
    print(long_poly)
    return total,long_poly,long_poly_and_TSD

def process_df(df):
    dict_of_loci = {}
    for row in df.itertuples():    
        #get genome to extract from
        if state == "query":
            samtools_genome = row.Ref
        else:
            samtools_genome = row.Query
        dist = 30
        if len(row.Age_TSD_Seq_right) >= 5:
            TSD_found = True
            TSD_len = len(row.Age_TSD_Seq_right)
            if row.RM_orientation == "Forward":
                left_bound= int(row.left_TSD_filled.split("\\")[1])+1
                #print(f"left bound of variant is {left_bound}")
                TSD_coords = row.right_TSD_filled.split("\\")
                poly_coords_left = int(TSD_coords[0])-dist
                poly_coords_right = int(TSD_coords[0])-1
                if poly_coords_left < left_bound:
                    poly_coords_left = left_bound
                head,sequence = samtools_extract(samtools_genome,row.chr,poly_coords_left,poly_coords_right, "")

                #print(head,sequence)
                #max_a = process_sequence(sequence)

            elif row.RM_orientation == "Reverse":
                right_bound = int(row.right_TSD_filled.split("\\")[0])-1
                TSD_coords = row.left_TSD_filled.split("\\")
                poly_coords_left = int(TSD_coords[1])+1
                poly_coords_right = int(TSD_coords[1])+dist
                if poly_coords_right > right_bound:
                    poly_coords_right = right_bound
                head,sequence = samtools_extract(samtools_genome,row.chr,poly_coords_left,poly_coords_right, "-i")
                #print(head,sequence)
            else:
                continue
        else:
            TSD_found = False
            TSD_len = len(row.Age_TSD_Seq_right)
            if row.RM_orientation == "Forward":
                left_bound= int(row.Refined_Filled_start)
                #TSD_coords = row.right_TSD_filled.split("\\")
                poly_coords_left = int(row.Refined_Filled_end)-(dist-1)
                poly_coords_right = int(row.Refined_Filled_end)
                if poly_coords_left < left_bound:
                    poly_coords_left = left_bound
                head,sequence = samtools_extract(samtools_genome,row.chr,poly_coords_left,poly_coords_right, "")

                #print(head,sequence)
                #max_a = process_sequence(sequence)

            elif row.RM_orientation == "Reverse":
                right_bound = int(row.Refined_Filled_end)
                TSD_coords = row.left_TSD_filled.split("\\")
                poly_coords_left = int(row.Refined_Filled_start)
                poly_coords_right = int(row.Refined_Filled_start)+(dist-1)
                if poly_coords_right > right_bound:
                    poly_coords_right = right_bound
                head,sequence = samtools_extract(samtools_genome,row.chr,poly_coords_left,poly_coords_right, "-i")
                #print(head,sequence)
            else:
                continue
        max_a = process_sequence(sequence)
        if "N" in row.Adjacent_Poly:
            poly_len = 0
        else:
            split_poly = row.Adjacent_Poly.split("\\")
            poly_len = int(split_poly[1])-int(split_poly[0])+1
        dict_of_locus = {'seq':sequence, 'max':max_a, 'TSD_found':TSD_found,'TSD_len':TSD_len, "Adjacent_Poly_length":poly_len,"RM_orientation":row.RM_orientation}
        dict_of_loci[f"{row.chr}_{row.Filled_start}_{row.Filled_end}"] = dict_of_locus
    #print(dict_of_loci)
    total, long_poly, long_poly_and_TSD = process_dict(dict_of_loci)
    print(f"Total number of loci is {len(df)}. Total number of loci with a unique orientation {total}. Loci with poly(A) at least 10bp adjacent to the TSD or at there is at least one segment of 15bp which is comprised of at least 10 As within {dist} of the TSD if at least 5bp or the boundary of the insertion if not {long_poly}. The previous, but only variants that also are have a 10bp TSD {long_poly_and_TSD}.")
    return dict_of_loci

In [3]:
directory = "/nfs/turbo/umms-jmkidd/matt-projects/Former_KiddLabScratch_Data/inter-genome_comparisons/Aligning_Using_2.26/Revised_All_Canines_List_2025_08_06"


filled_canine = "Mischka"
empty_sample = "mCanlor"
state = "query"
#TE = ["LINE","SINE"]
current_processing_file = f"{directory}/{filled_canine}_{empty_sample}_SV_{state}_filled_intersect_with_LINEs_processed_filtered.bed"
df_line = process_file(current_processing_file)
print(df_line.columns)
dict_of_loci_L1 = process_df(df_line)


current_processing_file = f"{directory}/{filled_canine}_{empty_sample}_SV_{state}_filled_intersect_with_SINEs_processed_filtered.bed"
df_sine = process_file(current_processing_file)
print(df_sine.columns)
dict_of_loci_SINE = process_df(df_sine)

(1962, 38)
Index(['chr', 'Filled_start', 'Filled_end', 'Empty_start', 'Empty_end',
       'Age_Insertion_Found', 'Age_TSD_Seq_empty', 'Age_TSD_Seq_left',
       'Age_TSD_Seq_right', 'Age_orientation', 'Interior_TE_dist', 'Prefix',
       'Query', 'RM_Class', 'RM_name', 'RM_orientation', 'RM_subfamilies',
       'Ref', 'Reference_orientation', 'Refined_Empty_end',
       'Refined_Empty_left_bound', 'Refined_Empty_right_bound',
       'Refined_Empty_start', 'Refined_Filled_end',
       'Refined_Filled_left_bound', 'Refined_Filled_right_bound',
       'Refined_Filled_start', 'TE_extends', 'age_endo', 'age_flank',
       'empty_coords', 'left_TSD_empty', 'left_TSD_filled', 'mismatch_TSD',
       'poly_a_list', 'right_TSD_empty', 'right_TSD_filled', 'Adjacent_Poly'],
      dtype='object')
1866
1633
Total number of loci is 1962. Total number of loci with a unique orientation 1866. Loci with poly(A) at least 10bp adjacent to the TSD or at there is at least one segment of 15bp which is compris

In [4]:
total = 0
long_poly = 0
def restrictive_processing(dictionary):
    Both = 0
    TSD = 0
    Poly = 0
    Neither = 0
    Neither_dict = {}
    for key in dictionary.keys():
        subdict = dictionary[key]
        if subdict['TSD_len'] >= 10 and subdict["Adjacent_Poly_length"] >= 10:
            Both +=1
        elif subdict['TSD_len'] >= 10:# and subdict['Adjacent_Poly_length'] != 0:
            TSD +=1
        elif subdict["Adjacent_Poly_length"] >= 10:
            Poly +=1
        else:
            Neither +=1
            Neither_dict[key] = subdict
    return_dict = {"Both":Both,'TSD':TSD,'Poly':Poly,"Neither":Neither}#,"Neither_Dict":Neither_dict}
    return return_dict
def permissive_processing(dictionary):
    Both = 0
    TSD = 0
    Poly = 0
    Neither = 0
    Neither_dict = {}
    for key in dictionary.keys():
        subdict = dictionary[key]
        if subdict['TSD_len'] >= 7 and (subdict["Adjacent_Poly_length"] >= 10 or subdict['max'] >= 10):
            Both +=1
        elif subdict['TSD_len'] >= 7:# and subdict['Adjacent_Poly_length'] != 0:
            TSD +=1
            #print(key,subdict)
        elif subdict["Adjacent_Poly_length"] >= 10 or subdict['max'] >= 10:
            Poly +=1
        else:
            Neither +=1
            Neither_dict[key] = subdict
    return_dict = {"Both":Both,'TSD':TSD,'Poly':Poly,"Neither":Neither}#,"Neither_Dict":Neither_dict}
    return return_dict



In [5]:
restrictive_dict = restrictive_processing(dict_of_loci_L1)
print(restrictive_dict)
permissive_dict = permissive_processing(dict_of_loci_L1)
print(permissive_dict)

{'Both': 866, 'TSD': 523, 'Poly': 217, 'Neither': 260}
{'Both': 1404, 'TSD': 98, 'Poly': 315, 'Neither': 49}


In [6]:
restrictive_dict = restrictive_processing(dict_of_loci_SINE)
print(restrictive_dict)
permissive_dict = permissive_processing(dict_of_loci_SINE)
print(permissive_dict)

{'Both': 7629, 'TSD': 4540, 'Poly': 843, 'Neither': 1521}
{'Both': 12625, 'TSD': 114, 'Poly': 1601, 'Neither': 193}


In [7]:
def parse_Hallmarks_in_dataframe(df):
    print(df.shape)
    TSD,Poly_A,Both,Neither,Both_Auto,TSD_Auto = 0,0,0,0,0,0
    min_TSD_len = 10
    min_Poly_A_len = 10
    endos = []
    short_endo = []

    for row in df.itertuples():
        
        #print(Adjacent_Poly)
        if "Multiple" in row.RM_orientation:
            continue
        if "N" in row.Adjacent_Poly:
            Adjacent_Poly = "N/A"
        else:
            Adjacent_Poly = row.Adjacent_Poly.split("\\")
            Adjacent_Poly[0] = int(Adjacent_Poly[0])
            Adjacent_Poly[1] = int(Adjacent_Poly[1])
        if len(row.Age_TSD_Seq_right) >= min_TSD_len and row.Age_TSD_Seq_right != "N/A" and Adjacent_Poly != "N/A" and Adjacent_Poly[1]-Adjacent_Poly[0]+1 >= min_Poly_A_len and row.RM_orientation != "Multiple": #has a TSD and a poly a
            Both +=1
            if row.chr != "chrX":
                Both_Auto +=1
        elif len(row.Age_TSD_Seq_right) >= min_TSD_len and row.Age_TSD_Seq_right != "N/A":# and Adjacent_Poly != "N/A":
            TSD +=1
            if row.chr != "chrX":
                TSD_Auto +=1
        elif Adjacent_Poly != "N/A" and Adjacent_Poly[1]-Adjacent_Poly[0]+1 >= min_Poly_A_len:
            Poly_A +=1
        else:
            Neither +=1
        if row.age_endo == "N/A":
            pass
        #elif  len(row.Age_TSD_Seq_left) >= 5 and  len(row.Age_TSD_Seq_left) < min_TSD_len:
        #    short_endo.append(row.age_endo.lower())
        elif len(row.Age_TSD_Seq_left) >= min_TSD_len:
            endos.append(row.age_endo.lower())
    return TSD,Poly_A,Both,Neither
parse_Hallmarks_in_dataframe(df_sine)

(14543, 38)


(4540, 843, 7629, 1521)

In [8]:
dictionary = dict_of_loci_L1
for item in dictionary.keys():
    if dictionary[item]['max'] >= 10 and dictionary[item]['Adjacent_Poly_length'] < 8:
        print(dictionary[item],item)

{'seq': 'aaatttattaaaaataaaatgaaataaaat', 'max': 12, 'TSD_found': True, 'TSD_len': 10, 'Adjacent_Poly_length': 4, 'RM_orientation': 'Reverse'} chr1_24850371_24850626
{'seq': 'aaaaaaaaaaaaataaataaataaataaaa', 'max': 14, 'TSD_found': True, 'TSD_len': 13, 'Adjacent_Poly_length': 4, 'RM_orientation': 'Reverse'} chr1_29809260_29810695
{'seq': 'aaaaaaaaaaaaaaaaaaaaaaaataaaac', 'max': 15, 'TSD_found': False, 'TSD_len': 3, 'Adjacent_Poly_length': 4, 'RM_orientation': 'Reverse'} chr1_30077510_30084105
{'seq': 'accaataaaaaaaaaaaaataagaaaaaaa', 'max': 14, 'TSD_found': True, 'TSD_len': 16, 'Adjacent_Poly_length': 7, 'RM_orientation': 'Forward'} chr1_44164651_44165440
{'seq': 'taaattgaacaccaataaaaaataaattaa', 'max': 11, 'TSD_found': True, 'TSD_len': 11, 'Adjacent_Poly_length': 3, 'RM_orientation': 'Reverse'} chr1_51460056_51460130
{'seq': 'ataccttcaggaaaaaaaaataaaataaaa', 'max': 13, 'TSD_found': True, 'TSD_len': 17, 'Adjacent_Poly_length': 4, 'RM_orientation': 'Forward'} chr1_51601447_51602066
{'se

{'seq': 'aaaaagaaaaagaaaaacaaaacaaaacaa', 'max': 13, 'TSD_found': True, 'TSD_len': 14, 'Adjacent_Poly_length': 4, 'RM_orientation': 'Forward'} chr19_45689206_45691949
{'seq': 'ctgaggaaaaaagagaaaaacaattaagat', 'max': 12, 'TSD_found': True, 'TSD_len': 14, 'Adjacent_Poly_length': 2, 'RM_orientation': 'Forward'} chr19_46925636_46925716
{'seq': 'ccaataaaaaattaaaaaaataaaataaaa', 'max': 13, 'TSD_found': True, 'TSD_len': 17, 'Adjacent_Poly_length': 4, 'RM_orientation': 'Forward'} chr19_53625880_53626261
{'seq': 'tgaacaccaataaaaaaataaattaaaaaa', 'max': 12, 'TSD_found': True, 'TSD_len': 13, 'Adjacent_Poly_length': 6, 'RM_orientation': 'Forward'} chr2_7904421_7905479
{'seq': 'ttgaacaccaataaaaaataaattaaaaaa', 'max': 12, 'TSD_found': True, 'TSD_len': 14, 'Adjacent_Poly_length': 6, 'RM_orientation': 'Reverse'} chr2_9539453_9540001
{'seq': 'tgaacaccaataaaaaataaattaaaaaaa', 'max': 12, 'TSD_found': True, 'TSD_len': 16, 'Adjacent_Poly_length': 7, 'RM_orientation': 'Reverse'} chr2_13768795_13768863
{'seq